In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import pickle
from collections import Counter, defaultdict
from itertools import combinations

import numpy as np
import scipy as sp
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import linalg
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

import numpy as np
from sklearn.manifold import TSNE
import glob

In [2]:
fnames = glob.glob("/data/shruti/conspirecovery/reddit/initial/network/subreddit_linkshare00000*.gz")
lstframe = []
for f in fnames:
    temp_F = pd.read_csv(f, header=0)
    lstframe.append(temp_F)

In [3]:
frame = pd.concat(lstframe)
frame.head()

,url,f0_
0,https://fortheloveofbands.com/2019/01/09/how-t...,"WeAreTheMusicMakers,spotify"
1,https://i.redd.it/aia31x84zxe21.jpg,"crafts,StarWars"
2,https://www.rawstory.com/2018/08/watch-sacha-b...,"politics,milwaukee"
3,https://www.ndtv.com/allahabad-news/kumbh-shah...,"AutoNewspaper,NDTVauto"
4,https://themerkle.com/bitcoin-cash-price-tops-...,"CryptoCurrency,AllThingsCrypto"


In [4]:
nodes = defaultdict(int)
edges = defaultdict(int)

for idx, row in frame.iterrows():
    sublist = row['f0_'].split(",")
    for s in sublist:
        nodes[s]+=1
        
    for i in range(len(sublist)-1):

        for j in range(i+1, len(sublist)):
            etuple = sorted([sublist[i], sublist[j]])
            edges[(etuple[0], etuple[1])]+=1


In [5]:
df = pd.Series(edges).reset_index()   
df.columns = ['n1', 'n2', 'weight']   
df.head()

,n1,n2,weight
0,WeAreTheMusicMakers,spotify,151
1,StarWars,crafts,47
2,milwaukee,politics,166
3,AutoNewspaper,NDTVauto,120278
4,AllThingsCrypto,CryptoCurrency,12283


In [6]:
len(df)

12993457

In [7]:
df.to_csv("/data/shruti/conspirecovery/reddit/initial/network/adjacency_matrix_full.csv")

In [10]:
len(df.loc[df['weight']>10])

634354

In [11]:
alt_df = df.loc[df['weight']>10]

In [14]:
alt_df.loc[(alt_df['n1']=='climateskeptics')&(alt_df['n2']=='science')]

,n1,n2,weight
465050,climateskeptics,science,331


In [19]:
sublist = list(set(list(set(alt_df['n1'].tolist())) + list(set(alt_df['n2'].tolist()))))

In [20]:
len(sublist)

8653

In [21]:
countMat = np.zeros((len(sublist), len(sublist)), dtype=np.float)

<ipython-input-21-2dfb3f1f706a>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  countMat = np.zeros((len(sublist), len(sublist)), dtype=np.float)


In [22]:
for idx, row in alt_df.iterrows():
    n1_idx = sublist.index(row['n1'])
    n2_idx = sublist.index(row['n2'])
    
    countMat[n1_idx, n2_idx] = row['weight']
    countMat[n2_idx, n1_idx] = row['weight']
    

In [23]:
def get_pmi_matrix(count_matrix, alpha=0.50, positive_only=False):
    # for standard PPMI
    DD = count_matrix.sum()
    sum_over_contexts = np.array(count_matrix.sum(axis=1)).flatten()
    sum_over_words = np.array(count_matrix.sum(axis=0)).flatten()

    # for context distribution smoothing (cds)
    sum_over_words_alpha = sum_over_words ** alpha
    Pc_alpha_denom = np.sum(sum_over_words_alpha)

    row_indxs = []
    col_indxs = []
    pmi_dat_values = []  # pointwise mutual information

    for idxs in tqdm(
            zip(*count_matrix.nonzero()),
            total=count_matrix.nnz,
            desc='building ppmi matrix row,col,dat'
    ):
        (tok_word_indx, tok_context_indx) = idxs
        pound_wc = count_matrix[tok_word_indx, tok_context_indx]
        pound_w = sum_over_contexts[tok_word_indx]
        pound_c_alpha = sum_over_words_alpha[tok_context_indx]

        Pwc = pound_wc / DD
        Pw = pound_w / DD
        Pc_alpha = pound_c_alpha / Pc_alpha_denom

        pmi = np.log2(Pwc / (Pw * Pc_alpha))
        if positive_only:
            pmi = max(pmi, 0)
        # if not np.isfinite(pmi):
        #     pmi = 0

        row_indxs.append(tok_word_indx)
        col_indxs.append(tok_context_indx)
        pmi_dat_values.append(pmi)

    print('building ppmi matrix')
    return sparse.csr_matrix((pmi_dat_values, (row_indxs, col_indxs)))

In [24]:
PMImat = get_pmi_matrix(sparse.csr_matrix(countMat))

building ppmi matrix row,col,dat: 100%|██████████| 1268708/1268708 [00:21<00:00, 57794.24it/s]


building ppmi matrix


In [25]:
def get_svd_matrix(pmi_matrix, embedding_size=200,
                   svd_diag_exponent=0.5):
    uu, ss, vv = linalg.svds(pmi_matrix, embedding_size)

    print('ppmi size: {}'.format(pmi_matrix.shape))
    print('embedding size: {}'.format(embedding_size))
    print('uu.shape: {}'.format(uu.shape))
    print('ss.shape: {}'.format(ss.shape))
    print('vv.shape: {}'.format(vv.shape))

    svd_word_vecs = uu.dot(np.diag(ss ** svd_diag_exponent))
    print(svd_word_vecs.shape)
    return svd_word_vecs

In [26]:
svd_embedding_size=200

In [27]:
svdMat = get_svd_matrix(PMImat, embedding_size=svd_embedding_size)

ppmi size: (8653, 8653)
embedding size: 200
uu.shape: (8653, 200)
ss.shape: (200,)
vv.shape: (200, 8653)
(8653, 200)


In [28]:
svdMat.shape

(8653, 200)

In [29]:
subreddit_embeddings = pd.DataFrame()
subreddit_embeddings['subreddit'] = sublist
for i in range(svd_embedding_size):
    subreddit_embeddings[i] = svdMat[:,i]

<ipython-input-29-076fcf03ac84>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  subreddit_embeddings[i] = svdMat[:,i]


In [30]:
subreddit_embeddings.head()

,subreddit,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,farscape,0.038648,0.081657,-0.106310,-0.105179,0.000057,-0.099921,0.173904,0.047933,-0.027336,...,-0.027443,-0.015201,0.017366,0.027397,0.007033,-0.011706,0.007817,0.017413,0.001915,0.014115
1,androidthemes,-0.146577,0.284977,-0.090275,0.050807,-0.015957,0.229644,-0.032222,0.243167,-0.049421,...,-0.062112,0.381539,0.169453,-0.116552,-0.206418,-0.984866,0.503715,0.406589,0.306237,0.734388
2,ObjectiveC,-0.001053,0.190702,-0.214452,-0.091735,-0.099704,-0.044693,0.344500,-0.195712,0.134266,...,0.285250,0.146518,0.329720,0.135467,0.055125,-0.041450,0.116224,0.061342,-0.043951,0.046310
3,WilliamShatner,0.010307,0.037257,-0.046474,-0.100993,0.027722,-0.066837,0.095636,0.026013,-0.023057,...,-0.055693,-0.004484,-0.049241,0.087176,-0.004974,-0.003601,-0.023730,0.029373,-0.005159,0.049786
4,ExoLife,0.061143,0.028786,-0.060122,-0.116302,-0.004554,0.056193,0.107799,0.092913,0.024393,...,0.103177,-0.004551,0.187006,0.168277,0.013686,0.008519,0.021653,0.008250,-0.097239,0.068467


In [31]:
subreddit_embeddings.to_csv("/data/shruti/conspirecovery/reddit/initial/network/subreddit_embeddings.csv")

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
from scipy.spatial.distance import cosine

In [45]:
subb = 'ClimateCrisis'
climate_similarity = pd.DataFrame(columns=['subreddit', 'similarity'])
climate_index = sublist.index(subb)
climate_vector = svdMat[climate_index, :].reshape(-1, 1)
for s in sublist:
    s_index = sublist.index(s)
    s_vector = svdMat[s_index, :].reshape(-1, 1)
    coss = cosine(climate_vector, s_vector)
    climate_similarity = climate_similarity.append({'subreddit':s, 'similarity':coss}, ignore_index=True)
    
climate_similarity.sort_values(by=['similarity']).head(50)

,subreddit,similarity
4309,ClimateCrisis,0.000000
5195,climatepolicy,0.125806
5436,climate_discussion,0.136141
7257,SunriseMovement,0.149859
3476,FridaysForFuture,0.169089
5317,GlobalWarming,0.187310
5461,climate_science,0.188217
294,StopFossilFuels,0.190882
6666,ClimateVideos,0.201090
2409,climateconsequences,0.201090


In [42]:
climate_similarity.sort_values(by=['similarity']).head(50)

,subreddit,similarity
5914,climateskeptics,0.000000
5317,GlobalWarming,0.222368
6484,climatechange,0.251667
8005,GlobalClimateChange,0.294969
5461,climate_science,0.297624
488,LockCarbon,0.328524
5302,ClimateHeadlineFraud,0.338298
294,StopFossilFuels,0.346954
911,climate,0.352044
5195,climatepolicy,0.359794
